In [5]:
tickerStr = "AAPL"

In [2]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import load_model
import os
import yfinance as yf

In [6]:
model = load_model(os.path.join("..","static","models",f"{tickerStr}.h5"))

In [ ]:
ticker = yf.Ticker(tickerStr)
df = ticker.history(period='5y')['Close'].values
df = df.reshape(-1, 1)
dataset_train = np.array(df[:int(df.shape[0]*0.8)])
dataset_test = np.array(df[int(df.shape[0]*0.8)-50:])
scaler = MinMaxScaler(feature_range=(0,1))
dataset_train = scaler.fit_transform(dataset_train)

def create_my_dataset(df):
    x = []
    y = []
    for i in range(50, df.shape[0]):
        x.append(df[i-50:i,0])
        y.append(df[i,0])
    x = np.array(x)
    y = np.array(y)
    return x,y

x_train, y_train = create_my_dataset(dataset_train)
x_test, y_test = create_my_dataset(dataset_test)

#reshaping for LSTM
x_train = np.reshape(x_train, (x_train.shape[0],x_train.shape[1],1))
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1],1))

predictions = model.predict(x_test)
predictions = scaler.inverse_transform(predictions)

predictions_prepended = [0 for i in range(0,df.shape[0]-len(predictions))]
predictions_prepended.extend(predictions.flatten().tolist())
graph1 = pd.DataFrame({
    "closing_prices": df.flatten().tolist(),
    "predicted_prices": predictions_prepended,
})
graph1.to_csv('graph_1.csv',index_label="Index")